In [4]:
# Load the advice dataset
with open("../data/heyfromthefuture/filtered.txt", "r") as f: lines = f.readlines()
lines = [line[:-2] for line in lines] # removes the newlines at the end

In [2]:
!pip3 install tensorflow_text>=2.0.0rc0

In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

# Load the model
module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-qa/3')

In [6]:
# Encode advice as responses
advice = tf.constant(lines)
advice_embeddings = module.signatures['response_encoder'](
    input=advice,context=advice) # Not sure what context is supposed to be but
                                 # putting the same thing for both seems to work

In [7]:
# Returns a sorted tuple list with advice and
# how "good" they are for the given query
def ask(query):
  # Encode the question (or more aptly an array of the single function)
  question_embeddings = module.signatures['question_encoder'](tf.constant([query]))

  # Get an array of floats for how "good" each adivce is for this query
  # We can do this via dot product because math
  # (and this is secretly a simmilarity problem)
  weights = np.inner(question_embeddings['outputs'],
                     advice_embeddings['outputs'])[0] # [0] because just one query

  # Returns a sorted tuple list with advice and
  # how "good" they are for the given query
  return sorted([(lines[i], weights[i]) for i in range(len(lines))], key=lambda x: -x[1])

In [21]:
# What question do you ask! This is simmilar to what our final user would enter
query = "Hey 21 year old. i am sad"
# Gets the top `n` responses
n = 10

# Does the do
top_results = ask(query)[:n]
for r in top_results:
    print(r[0])


Hey 21 year old, You're not behind everyone else. Don't be so worried about reaching the next step in life as soon as possible. Travel, explore, have fun. This is the easiest time in your life to be spontaneou
Hey 24 year old, love yourself and don't be sad over someone who left you. always have faith and wait for true lov
Hey 19 year old, Take pride in your hobbies they will keep you happy on bad day
Hey 21 year old, Meet people and engage in activities. It creates opportunities you even haven't imagined of. People are the most important things to nurtur
Hey 22 year old, take a deep breath, go for a long walk in the woods (or plant trees for a few months out west if you can), and realize that you can and should capitalize on your talents now, don't wait. get over your disappointment in the world and creat
Hey 23 year old, It's never too late to put yourself out there. Get on those dating apps, meet people, and most importantly have fu
Hey 21 year old, don't let the abundance of career